# Tree Based Methods

- For both regression and classification.
- These involve segmenting the predictor space into a number of simple regions.
- In order to make a prediction we typically use mean or mode of the training observation in the region to which it belongs.
- These set of splitting rules, used to segment the predictor space, can be summarized in a tree.
- Tree based methods are useful for interpretation.

## Regression Trees

- The regions are known as terminal nodes or leaves.
- The points along the tree where the predictor space splits are referred to as internal nodes.
- The segment of trees which connect the nodes are branches.

### Prediction via stratification of the Feature/Predictor space:

1. We divide the predictor space - i.e, the set of possible values of $X_1, X_2, ..$ into j distinct and non-overlapping regions $R_1, R_2, ...$.
2. For every observation that falls into region $R_j$ we make the same prediction, which is simply the mean of the response values for the training observations in $R_j$.

#### How to construct the regions ?

- The goal is to find regions which minimize the RSS (Residual Sum of Squares).

    $\sum_{j=1}^{J} \sum_{i \in R_j} (y_i - \hat{y_{R_j}})^2$

- Unfortunately it is computationally infeasible to consider every possible partition of the feature space into  boxes.
- For this reason, we use the top-down greedy approach, known as recursive binary splitting. 
- Best split is made at that particular step.

#### Binary Recursive splitting

- We first select the predictor $X_j$ and the cutpoint s such that splitting the predictor space into the regions ${X|X_j < s}$ and ${X|X_j >= s}$ leads to greatest possible reduction in RSS. 
- The notation ${X|X_j < s}$ means the region of predictor space in which $X_j$ takes on a value less than s.

- For any j and s, we define pair of half-planes as

    $R_1 (j,s) = {X|X_j < s}$ and $R_2 (j,s) = {X|X_j >= s}$

- And we seek the value of j and s to minimize the equation,

    $\sum_{i:x_i \in R_1(j,s)} (y_i - \hat{y_{R_1}})^2 + \sum_{i:x_i \in R_2(j,s)} (y_i - \hat{y_{R_2}})^2$

- Next we repeat the process, looking for best predictor and best cutpoint in order to split the data further to minimize the RSS within each of the resulting regions.
- However, this time, instead of splitting the whole predictor space, we split one of any two previously identified regions.
- Once the regions are created, we predict the response for a given test observation using the mean of the training observations in the region to which that observation belongs.

### Tree Pruning

- The above process may produce good predictions but is likely to overfit the data.
- One possible way is to build the tree so long as the decrease in the RSS due to each split exceeds certain threshold.
- This strategy may result in smaller trees but is too short-sighted since a seemingly worthless split early on tree might be followed by a good split(i.e it leads to large reduction in RSS). 
- A better strategy is to grow a very large tree and then 'prune' it back in order to obtain a subtree. Our goal is to select a subtree that leads to lower error rate(using cross-validation).
- However, estimating the cross-validation would be too cumbersome since there will be large number of possible subtrees.
- Rather than considering every subtree, we consider a sequence of trees indexed by a non-negative number $\alpha$.
- For each value of $\alpha$ there corresponds a subtree such that 
    
    $\sum_{m =1}^{|T|} \sum_{x_i \in R_m} (y_i - \hat{y_{R_m}})^2 + \alpha |T| $ 
    
   
- is as small as possible. $|T|$ indicates number of terminal nodes, $R_m$ is the rectangle (subset of predictor space) coresponding to the mth terminal node and $\hat{y_{R_m}}$ is the predicted response associated with $R_m$ - That is mean of the training observations $R_m$.
- When $\alpha = 0$, then subtree T simply equals $T_0$ 
- As we increase $\alpha$ from zero, branches get pruned from the tree in a nested and predictable fashion.

Algorithm: 

1. Use binary recursive splitting to grow a large tree.

2. Apply cost complexity pruning to the large tree to obtain a sequence of best subtrees as a function of $\alpha$.

3. Use k-fold cross-validation to choose $\alpha$. For each k = 1, 2...K:
    - Repeat steps 1 and 2 on all but kth fold.
    - Evaluate the mean squared prediction error 
    - Average the results for each value of $\alpha$ and pick one to minimize average error.
    
4. Return the subtree from step 2 that corresponds to choosen value of $\alpha$

## Classification Trees

- Very similar to regression trees, except that it is used to predict a qualitative response.
- Here, we predict that each observation belongs to most commonly occurring class of training observations in the region to which it belongs.
- We are also interested in class proportions.
- We use recursive binary splitting to grow a classification tree.
- We use classification error rate instead of RSS.

- Classification error rate is simply the fraction of training observations in that region that do not belong ot most common class:

    $E = 1 - max_{k} (\hat{p_{mk}})$
    
    
- Here $\hat{p_{mk}}$ represents the proportion of training observations in the mth region that are from kth class.

### Gini Index

- However it is not sufficiently sensitive for tree-growing.
- Another measure is 'Gini Index'

    $G = \sum_{k=1}^{K} \hat{p_{mk}}(1-\hat{p_{mk}})$
    
    
- It is a measure of total variance across k classes.

### Entropy

- An alternative to Gini index is Entropy. 

    $ D = -\sum_{k=1}^{K} \hat{p_{mk}} \log \hat{p_{mk}} $
    
- One can show that entropy will take on a value near zero if $\hat{p_{mk}}$'s are all near zero or near one.
- Therefore, entropy (like Gini index) will take on a small value if the mth node is pure.
- They are used to evaluate the quality of a particular split.When a data point is close to one class the entropy is low

- If all the data points belong to a single class, then there is no real uncertainity, there will be low entropy i/e $p_i$ is close to 0 or 1
- If the data points are evenly spread across the classes, there is a lot of uncertainity and hence high entropy i.e $p_i$'s will be far from 0 and 1.

In [2]:
# entropy 
def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p,2)
                for p in class_probabilities
                if p)          # ignore zero probabilities 

In [3]:
def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
           for count in Counter(labels).values()]

In [5]:
def data_entropy(labeled_data):
    labels = [label for _, in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

- If we partition our data S into subsets $S_1$, $S_2$ .. containing proportions $q_1...q_m$, then we compute the entropy of partition as a weighted sum: 

      $H = q_1 H(S_1) + ..... + q_m H(S_m)$

In [8]:
def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets
    subsets is a list of lists of labeled data(eg. true/false labels)"""
    
    total_count = sum(len(subset) for subset in subsets)
    
    return sum(data_entropy(subset) * len(subset) / total_count
              for subset in subsets)

In [9]:
# Partition data by each of attributes
def partition_by(inputs, attribute):
    """each input is a pair (attribute_dict, label).
    returns a dict : attribute_value -> inputs"""
    groups = defaultdict(list)
    for input in inputs:
        key = input[0][attribute] # get the value of the specified attribute
        groups[key].append(input) # then add this input to the correct list
    return groups

In [10]:
# Function which computes entropy for each partition
def partition_entropy_by(inputs, attribute):
    """computes the entropy corresponding to the given partition"""
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

In [13]:
# Choose partition with minimum entropy
for key in ['level', 'lang', 'tweets', 'phd']:
    print key, partition_entropy_by(inputs, key)

SyntaxError: invalid syntax (<ipython-input-13-82e87c26f4c5>, line 3)

In [ ]:
# Lowest entropy from 'level' so we make a subtree for each possible 'level' value
senior_inputs = [(input, label)
                for input, label in inputs if input['level'] == "Senior"]

for key in ['lang', 'tweets', 'phd']:
    print key, partition_entropy_by(senior_inputs. key)

## Bagging 

- The decision trees suffer from high variance.
- This means that if we split training data into 2 parts at random and fit a decion tree to both halves, the results that we get could be quite different.
- Boostrap aggregation or 'bagging' is a general purpose procedure for reducing the variance of a statistical learning method.
- Averaging a set of observations reduces variance.
- A natural way to reduce variance is to take many training sets from population.
- We could calculate $\hat{f^1}(x), \hat{f^2}(x)...$ using B training sets and average them in order to obtain a single low-variance statistical learning model.

    $\hat{f}_{avg}(x) = \frac{1}{B} \sum_{b=1}^{B}\hat{f}^b(x)$
    

- This is not practical because we generally do not have access to multiple training sets. Instead we can bootstrap by taking repeated samples from a single training set. 
- In this approach we generate B different bootstrapped training sets. We then train our model on bth bootstrapped training set to get $\hat{f^{*b}}(x)$ and finally average predictions to obtain:
    $\hat{f_{bag}(x)}= \frac{1}{B} \sum_{b=1}^{B}\hat{f^{b}}(x)$
    
   
- This is called bagging.

#### Bagging for regression

- To apply bagging to regression trees we construct B regression trees using B bootstrapped training sets and average the resulting predictions.
- These trees are grown deep and not pruned.
- Averaging the B trees reduces variance.

#### Bagging for classification

- Simplest way is to record the class of test observation predicted by bunch of B trees and take majority vote.

## Random Forests

- They provide an improvement over bagged tree by way of a small tweak that decorrelates the trees.
- As in bagging, we build a number of decision trees based on bootstrapping training samples.
- But when building these decision trees, each time a split in a tree is considered, a random sample of m predictors is chosen as split candidates from full set of p predictors.
- The split is allowed to use only one of those m predictors.
- A fresh sample of m predictors is taken at each split and typically we choose $ m = \sqrt{p}$.

- Main difference between Random forest and bagging is the choice of predictor size m.

## Boosting

- Another approach for improving the predictions resulting from a decision tree.
- Trees are grown sequentially. Each tree is grown using information from previously grown trees.
- Boosting does not involve bootstrap sampling, instead each tree is fit on a modified version of the orginal data set.

- Algorithm for boosting:

    1. Set the $\hat{f}(x) = 0$ and $r_i = y_i$ for all i in the training set.

    2. For b=1,2,...B repeat:
        1. Fit a tree $\hat{f^b}$ with d splits (d+1 terminal nodes) to the training data (X,r).
        2. Update $\hat{f}$ by adding in a shrunken version of the new tree:
    
            $\hat{f}(x) = \hat{f}(x) + \lambda \hat{f^b}(x)$
        
        3. Update the residuals
        
            $r_i = r_i - \lambda \hat{f^b}(x)$
        
    3. Output the boosted model

         $\hat{f}(x) = \sum_{b=1}^{B} \lambda \hat{f^b}(x)$

- The boosting approach learns slowly.
- We fit a tree to the residuals from the model rather than the outcome Y as response. We then add this new decision tree into the fitted function in order to update the residuals. 
- Each of these trees can be small with ust few terminal nodes determined by parameter 'd' in the algorithm.
- By fitting small trees to the residuals, we slowly improve $\hat{f}$ in areas where it does not perform well.
- The shrinkage parameter $\lambda$ slows the process down even further allowing more and different shaped trees to attack residuals. 
- The cosntruction of each tree depends strongly on trees that have already been grown (unlike in bagging). 

- Tuning parameters:
    1. Number of trees B: Boosting can overfit if B is too large.
    2. Shrinkage parameter $\lambda$ a small positive number. This controls the rate at which boosting learns. Typical values are 0.01 or 0.001.
    3. Number d of splits in each tree which controls the complexity of the boosted ensemble.